In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import time
import rstr
import random

from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from method.ours.feedback import get_global_feedback
from method.ours.utils import create_driver, get_xpath, interact_with_input

In [4]:
HEADLESS = False
URL = 'https://www.aircanada.com/ca/en/aco/home.html'

In [5]:
driver = create_driver(HEADLESS)
driver.get(URL)

In [6]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
form = driver.find_elements(By.TAG_NAME, 'form')[1]
form_xpath = get_xpath(driver, form)

inputs = form.find_elements(
    By.XPATH,
    f'{form_xpath}//input | {form_xpath}//textarea | {form_xpath}//select'
)

inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

In [7]:
tested = []

# Generate Base Values

In [8]:
fixed_rules = {
    'text': lambda: 'test',
    'email': lambda: 'test@test.com',
    'password': lambda: 'test',
    'number': lambda: '0',
    'range': lambda: '0',
    'date': lambda: '2020-01-01',
    'time': lambda: '00:00',
    'datetime-local': lambda: '2020-01-01T00:00',
    'week': lambda: '2020-W01',
    'month': lambda: '2020-01',
    'tel': lambda: '0123456789',
    'url': lambda: 'https://test.com',
    'color': lambda: '#000000',
    
    'boolean': lambda: True,
    'select': lambda _: 0,
}


random_rules = {
    'text': lambda: rstr.xeger(r'[A-Za-z0-9]{10}'),
    'email': lambda: rstr.xeger(r'[A-Za-z0-9]{10}') + '@' + rstr.xeger(r'[A-Za-z0-9]{10}') + '.com',
    'password': lambda: rstr.xeger(r'[A-Za-z0-9]{10}'),
    'number': lambda: random.randint(0, 1000),
    'range': lambda: random.randint(0, 1000),
    'date': lambda: rstr.xeger(r'[0-9]{4}') + '-' + rstr.xeger(r'[0-9]{2}') + '-' + rstr.xeger(r'[0-9]{2}'),
    'time': lambda: rstr.xeger(r'[0-9]{2}') + ':' + rstr.xeger(r'[0-9]{2}'),
    'datetime-local': lambda: rstr.xeger(r'[0-9]{4}') + '-' + rstr.xeger(r'[0-9]{2}') + '-' + rstr.xeger(r'[0-9]{2}') + 'T' + rstr.xeger(r'[0-9]{2}') + ':' + rstr.xeger(r'[0-9]{2}'),
    'week': lambda: rstr.xeger(r'[0-9]{4}') + '-W' + rstr.xeger(r'[0-9]{2}'),
    'month': lambda: rstr.xeger(r'[0-9]{4}') + '-' + rstr.xeger(r'[0-9]{2}'),
    'tel': lambda: rstr.xeger(r'[0-9]{10}'),
    'url': lambda: 'https://' + rstr.xeger(r'[A-Za-z0-9]{10}') + '.com',
    'color': lambda: '#' + rstr.xeger(r'[0-9A-Fa-f]{6}'),
    
    'boolean': lambda: True if random.randint(0, 1) == 0 else False,
    'select': lambda element: random.randint(0, len(Select(element).options) - 1),
}

In [9]:
def rule_based_value_generator(inputs, random=True):
    rules = random_rules if random else fixed_rules
    
    values = {}
    
    for element in inputs:
        tag = element.tag_name
        element_xpath = get_xpath(driver, element)
        input_type = element.get_attribute('type') or 'text'
        
        if tag == 'select':
            values[element_xpath] = rules['select'](element)
        
        elif input_type == 'checkbox' or input_type == 'radio':
            values[element_xpath] = rules['boolean']()
        
        else:
            values[element_xpath] = rules[input_type]()
    
    return values

In [10]:
def fill_form_with_values(xpaths, values):
    for element_xpath in xpaths:
        element = driver.find_element(By.XPATH, element_xpath)
        element_value = values[element_xpath]

        if element.get_attribute('type') in ['checkbox', 'radio']:
            continue

        try:
            interact_with_input(element, element_value)
        except Exception as e:
            print(e)
    
    submit_button = driver.find_element(By.XPATH, f'{form_xpath}//button[@type = "submit"]')
    submit_button.click()

In [11]:
base_values = rule_based_value_generator(inputs)

# Fuzz Input Values

In [12]:
fuzz_samples = {}

with open('fuzz-samples.json', 'r') as f:
    fuzz_samples = json.load(f)

In [13]:
for element_xpath in base_values.keys():
    driver.get(URL)
    time.sleep(1.5)
    
    new_values = dict(base_values)
    element = driver.find_element(By.XPATH, element_xpath)
    
    print(element.get_attribute('outerHTML'))
    print()
    
    if element.get_attribute('type') not in fuzz_samples:
        time.sleep(1)
        continue
    
    fuzz_sample = fuzz_samples[element.get_attribute('type')]
    
    for sample_name, sample_values in fuzz_sample.items():
        for sample_value in sample_values:
            driver.get(URL)
            time.sleep(1.5)
            
            new_values[element_xpath] = sample_value
            fill_form_with_values(base_values.keys(), new_values)
            diff = get_global_feedback(
                html,
                driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
            )
            tested.append({
                "fuzz_type": sample_name,
                "values": new_values,
                "changes": diff,
            })

<input type="radio" class="abc-form-element-main abc-form-element-radio-button ng-untouched ng-pristine ng-valid" id="bkmgFlights_tripTypeSelector_R" name="tripType" value="R" aria-disabled="false" aria-invalid="false" aria-checked="true">

<input type="radio" class="abc-form-element-main abc-form-element-radio-button ng-untouched ng-pristine ng-valid" id="bkmgFlights_tripTypeSelector_O" name="tripType" value="O" aria-disabled="false" aria-invalid="false" aria-checked="false">

<input type="radio" class="abc-form-element-main abc-form-element-radio-button ng-untouched ng-pristine ng-valid" id="bkmgFlights_tripTypeSelector_M" name="tripType" value="M" aria-disabled="false" aria-invalid="false" aria-checked="false">

<input type="checkbox" role="checkbox" class="abc-form-element-main abc-form-element-checkbox ng-untouched ng-pristine ng-valid" id="bkmgFlights_searchTypeToggle" aria-label="Book with Aeroplan points" aria-disabled="false" aria-required="false" aria-invalid="false" aria-che

<input class="abc-form-element-input abc-form-element-main text-transform-none ng-untouched ng-pristine ng-valid" type="text" placeholder="DD/MM" id="bkmgFlights_travelDates_1-formfield-1" name="bkmgFlights_travelDates_1-formfield-1" aria-disabled="false" aria-invalid="false" aria-label="Departure" aria-describedby="bkmgFlights_travelDates_1DatepickerSrHelperText" autocorrect="off" spellcheck="false" autocomplete="off">

Message: unknown error: ChromeDriver only supports characters in the BMP
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x000000010ae8d6b8 chromedriver + 4937400
1   chromedriver                        0x000000010ae84b73 chromedriver + 4901747
2   chromedriver                        0x000000010aa42616 chromedriver + 435734
3   chromedriver                        0x000000010aabe01b chromedriver + 942107
4   chromedriver                        0x000000010aa7f7a4 chromedriver + 685988
5   chromedriver                        0x0

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[1]/DIV[1]/INPUT[1]"}
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x000000010ae8d6b8 chromedriver + 4937400
1   chromedriver                        0x000000010ae84b73 chromedriver + 4901747
2   chromedriver                        0x000000010aa42616 chromedriver + 435734
3   chromedriver                        0x000000010aa85e0f chromedriver + 712207
4   chromedriver                        0x000000010aa860a1 chromedriver + 712865
5   chromedriver                        0x000000010aac79a4 chromedriver + 981412
6   chromedriver                        0x000000010aaaa03d chromedriver + 860221
7   chro

In [14]:
with open('', tested)

[{'fuzz_type': 'empty',
  'values': {'//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[1]/DIV[1]/INPUT[1]': True,
   '//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[2]/DIV[1]/INPUT[1]': True,
   '//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[3]/DIV[1]/INPUT[1]': True,
   '//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIG